# Running trajectory 

The target of this file is to provide step by step implementation on how to launch a visualisation of a given trajectory on H1 robot.
The trajectory is given on a **.npy** file. It is supposed to perform a **squat movement**.

I had some issues with joint order, which was different that the one defined on the urdf.


In [6]:
# import robot_loader from scripts
import pinocchio as pin
import os
import time
import numpy as np
from scripts.robot_loader import launch_visualization

# I choose to make abstraction on this part as it is not really interesting. See the specific file for more details
# load model on Meshcat
model, collision_model, visual_model, robot_visualizer, viz = launch_visualization()


--- Searching for robot paths ---
Detected project root directory: /home/cpene/Documents/robot_playground
Asset paths verified.
Detected urdf file: /home/cpene/Documents/robot_playground/robot_models/biped_assets/biped_assets/models/h12/h12_12dof.urdf
Detected mesh directory: /home/cpene/Documents/robot_playground/robot_models/biped_assets/biped_assets/models/h12
Launching MeshCat...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7005/static/
Adding a grid to represent the ground.
Loading robot model into Pinocchio...
Pinocchio model loaded successfully.

Environment is ready !


In [2]:
# Display the robot on in initial position, with his feets close to the ground
q0 = pin.neutral(model)
q0[2] = 1.03

robot_visualizer.display(q0)

In [3]:
# This part is abstracted, and resolve the problem that trajectory inputs data doesn't have the same joints order as pinocchio
# see specific file for more details
from scripts.trajectory_utils import convert_trajectory_to_pinocchio_format

# Path to trajectory datas 
trajectory_file = os.path.join("data", "h1v2_squat_fixed_ankle_states.npy")
raw_trajectory = np.load(trajectory_file)

# call conversion tool
q_trajectory, v_trajectory = convert_trajectory_to_pinocchio_format(raw_trajectory)

Raw trajectory converted and separated into q and v trajectories.


In [4]:
# At some point, I needed to know the dimension of the urdf. After checking,
# some modification on the urdf were needed to match dimension from the trajectory
print(f"Robot : {model.name}")
print(f"robot.nq = {model.nq}.")
print(f"robot.nv = {model.nv}.")

q_dim_match = (model.nq == q_trajectory.shape[1])
v_dim_match = (model.nv == v_trajectory.shape[1])

if q_dim_match and v_dim_match:
    print("\nDimensions of both q and v trajectories match the robot model. Great!")
else:
    print("\n" + "="*50)
    print("ERROR: Dimension mismatch between robot model and trajectory!")
    # On spécifie quelle dimension est incorrecte
    if not q_dim_match:
        print(f"  -> Configuration (q) ERROR: Model nq={model.nq}, but trajectory has {q_trajectory.shape[1]} columns.")
    if not v_dim_match:
        print(f"  -> Velocity (v) ERROR: Model nv={model.nv}, but trajectory has {v_trajectory.shape[1]} columns.")
    print("="*50)
    raise ValueError("Dimension mismatch detected. Please check your URDF or data conversion script.")

Robot : h1_5
robot.nq = 34.
robot.nv = 33.

Dimensions of both q and v trajectories match the robot model. Great!


In [5]:
# Launch the simulation

print("Starting animation...")

for q in q_trajectory:

    # Send q to robot visualization
    robot_visualizer.display(q)

    # dt is 0.033333333 s
    time.sleep(0.033)

print("\nAnimation end.")

Starting animation...

Animation end.
